# SETUP

## imports

In [56]:
import string
import numpy as np
from itertools import cycle
import requests
import collections
from collections import deque
from pprint import pprint
import operator
from time import sleep
import itertools
import re
import functools
from functools import reduce
from dataclasses import dataclass
from copy import deepcopy
import networkx as nx
import matplotlib.pyplot as plt
import intcomp

## constants

In [57]:
lowercase = string.ascii_lowercase
uppercase = string.ascii_uppercase

## helpers

In [58]:
def get_level_input(lvl_num):
    with open(f"advent_inputs/{lvl_num}.txt") as f:
        level_input=f.read()
        return level_input
    
def print_result(answer):
    pprint("RESULT: "+str(answer))
    print()
    pprint("TIME"+"."*60)
    
class StopExecution(Exception):
    def _render_traceback_(self):
        pass

# 1

## setup

In [17]:
module_weights = get_level_input("01").splitlines()
module_weights = list(map(int, module_weights))

## part one

In [18]:
%%time
total_weight = sum([x//3 - 2 for x in module_weights])
print_result(total_weight)

'RESULT: 3432671'

'TIME............................................................'
CPU times: user 531 µs, sys: 424 µs, total: 955 µs
Wall time: 814 µs


## part two

In [6]:
%%time

def weight_of_weight_gen(m_weight):
    while m_weight>0:
        yield m_weight
        m_weight = m_weight//3 - 2
    
def get_req_fuel(m_weight): 
    return sum([i for i in weight_of_weight_gen(m_weight)][1:])
    
total_weight = sum([get_req_fuel(x) for x in module_weights])
print_result(total_weight)

'RESULT: 5146132'

'TIME............................................................'
CPU times: user 402 µs, sys: 46 µs, total: 448 µs
Wall time: 429 µs


# 2

## setup

In [48]:
OPCODE = 0
INPUT_1 = 1
INPUT_2 = 2
OUTPUT = 3
WINDOW = 4

comp_string = get_level_input("02")
comp_string = list(map(int, comp_string.split(',')))
print(comp_string)
# comp_string = [1,9,10,3,2,3,11,0,99,30,40,50]


[1, 0, 0, 3, 1, 1, 2, 3, 1, 3, 4, 3, 1, 5, 0, 3, 2, 6, 1, 19, 1, 5, 19, 23, 1, 13, 23, 27, 1, 6, 27, 31, 2, 31, 13, 35, 1, 9, 35, 39, 2, 39, 13, 43, 1, 43, 10, 47, 1, 47, 13, 51, 2, 13, 51, 55, 1, 55, 9, 59, 1, 59, 5, 63, 1, 6, 63, 67, 1, 13, 67, 71, 2, 71, 10, 75, 1, 6, 75, 79, 1, 79, 10, 83, 1, 5, 83, 87, 2, 10, 87, 91, 1, 6, 91, 95, 1, 9, 95, 99, 1, 99, 9, 103, 2, 103, 10, 107, 1, 5, 107, 111, 1, 9, 111, 115, 2, 13, 115, 119, 1, 119, 10, 123, 1, 123, 10, 127, 2, 127, 10, 131, 1, 5, 131, 135, 1, 10, 135, 139, 1, 139, 2, 143, 1, 6, 143, 0, 99, 2, 14, 0, 0]


## part one

In [46]:
%%time

comp_string_prog = comp_string[:]
comp_string_prog[1] = 12
comp_string_prog[2] = 2
res = intcomp.run_computer(comp_string_prog, [])
print_result(comp_string_prog[0])

'RESULT: 5290681'

'TIME............................................................'
CPU times: user 1.64 ms, sys: 805 µs, total: 2.45 ms
Wall time: 1.83 ms


## part two 

In [55]:
%%time

TARGET = 19690720
NOUN_POS = 1
VERB_POS = 2
    
def find_noun_verb(comp_string):
    for noun in range(100):
        for verb in range(100):
            comp_string_copy = comp_string[:]
            comp_string_copy[NOUN_POS] = noun
            comp_string_copy[VERB_POS] = verb
            res = intcomp.run_computer(comp_string_copy, []) 
            if comp_string_copy[0] == -1: continue 
            if(comp_string_copy[0] == TARGET): return noun,verb

noun, verb = find_noun_verb(list(comp_string))

print_result(100 * noun + verb)

'RESULT: 5741'

'TIME............................................................'
CPU times: user 1.59 s, sys: 9.47 ms, total: 1.6 s
Wall time: 1.61 s


# 3

## setup

In [56]:
wires_input = get_level_input("03").splitlines()
wires_input = [x.split(',') for x in wires_input]

X = 1
Y = 0
CENTER_POINT = (0,0)

def md(p1, p2):
    return (abs(p1[X]-p2[X]) + abs(p1[Y]-p2[Y]))

## part one

In [88]:
%%time

def generate_wire(instructions):
    wire = [CENTER_POINT]
    for x in instructions:
        cur_point = wire[-1]
        wire += {
         'U': [(cur_point[Y]-i, cur_point[X]) for i in range(1, int(x[1:])+1)],
         'D': [(cur_point[Y]+i, cur_point[X]) for i in range(1, int(x[1:])+1)],
         'L': [(cur_point[Y], cur_point[X]-i) for i in range(1, int(x[1:])+1)],
         'R': [(cur_point[Y], cur_point[X]+i) for i in range(1, int(x[1:])+1)]
        }[x[0]]
    return wire
                                  
    
wires = [set(generate_wire(w_i))-{CENTER_POINT} for w_i in wires_input]
res = min([md(x, CENTER_POINT) for x in set.intersection(*wires)])
print_result(res)

'RESULT: 225'

'TIME............................................................'
CPU times: user 271 ms, sys: 5.11 ms, total: 276 ms
Wall time: 276 ms


## part two

In [142]:
%%time

wires = [generate_wire(w_i) for w_i in wires_input]
intersections = set.intersection(*[set(wire) for wire in wires])-{CENTER_POINT}
res = min([wires[0].index(x)+wires[1].index(x) for x in intersections])
print_result(res)

'RESULT: 35194'

'TIME............................................................'
CPU times: user 358 ms, sys: 4.02 ms, total: 362 ms
Wall time: 362 ms


# 4

## setup

In [140]:
r_low, r_high = map(int, get_level_input("04").split("-"))

def check_value(val, functions):
    num_list = [int(i) for i in list(str(val))]
    return (all(f(num_list) for f in functions))

## part one

In [143]:
%%time

def check_increase(l):
    return all(l[i] <= l[i+1] for i in range(len(l)-1))

def check_double(l):
    return any(l[i] == l[i+1] for i in range(len(l)-1))

num_poss_pass = 0
for num in range(r_low, r_high+1):
    if(check_value(num, [check_increase, check_double])): num_poss_pass+=1

print_result(num_poss_pass)

'RESULT: 1694'

'TIME............................................................'
CPU times: user 1.67 s, sys: 4.64 ms, total: 1.67 s
Wall time: 1.67 s


## part two 

In [144]:
%%time

def check_increase(l):
    return all(l[i] <= l[i+1] for i in range(len(l)-1))

def check_ungrouped_double(l):
    l = [-1] + l +[-1]
    return any(l[i-1] != l[i] == l[i+1] != l[i+2] for i in range(1, len(l)-1))

num_poss_pass = 0
for num in range(r_low, r_high+1):
    num_list = [int(i) for i in list(str(num))]
    if(check_value(num, [check_increase, check_ungrouped_double])): num_poss_pass += 1

print_result(num_poss_pass)

'RESULT: 1148'

'TIME............................................................'
CPU times: user 2.58 s, sys: 5.62 ms, total: 2.58 s
Wall time: 2.58 s


# 5

## setup

In [26]:
comp_string = get_level_input("05")
comp_string = list(map(int, comp_string.split(',')))

## part one

In [27]:
%%time

outputs = intcomp.run_computer(comp_string[:], [1])
print_result(outputs[-1])

0
0
0
0
0
0
0
0
0
10987514
'RESULT: 10987514'

'TIME............................................................'
CPU times: user 1.04 ms, sys: 316 µs, total: 1.35 ms
Wall time: 1.06 ms


In [29]:
%%time

outputs = intcomp.run_computer(comp_string[:], [5])
print_result(outputs[0])

14195011
'RESULT: 14195011'

'TIME............................................................'
CPU times: user 672 µs, sys: 79 µs, total: 751 µs
Wall time: 706 µs


# 6

## setup 

In [30]:
orbits = get_level_input("06").splitlines()
orbits = [orbit.split(')') for orbit in orbits]

## part one

In [31]:
%%time

orbit_graph = nx.DiGraph()
for orbit in orbits:
    orbit_graph.add_edge(*orbit)

orbit_checksum = sum([len(nx.descendants(orbit_graph, node)) for node in orbit_graph.nodes()])

print_result(orbit_checksum)

'RESULT: 144909'

'TIME............................................................'
CPU times: user 386 ms, sys: 4.07 ms, total: 390 ms
Wall time: 391 ms


## part two 

In [32]:
TARGET = "SAN"
START = "YOU"

orbit_graph = nx.Graph()
for orbit in orbits:
    orbit_graph.add_edge(*orbit)

START_SR = list(orbit_graph.neighbors(START))[0]
SAN_SR = list(orbit_graph.neighbors(TARGET))[0]

print_result(nx.shortest_path_length(orbit_graph, str(START_SR), str(SAN_SR)))
    

'RESULT: 259'

'TIME............................................................'


# 7

## setup 

In [59]:
amp_prog = get_level_input("07")
amp_prog = list(map(int, amp_prog.split(',')))
#amp_prog = [3,23,3,24,1002,24,10,24,1002,23,-1,23,101,5,23,23,1,24,23,23,4,23,99,0,0] #[3,15,3,16,1002,16,10,16,1,16,15,15,4,15,99,0,0]
NUM_AMPS = 5

# def run_computer(comp_string, c_i):
#     i=0
#     last_output = 0
#     while(True):
#         opcode_and_settings = "0000"+str(comp_string[i])
#         opcode = int(opcode_and_settings[-2:])
#         if(opcode == END): return comp_string, last_output
#         input_modes = [int(opcode_and_settings[-3]), int(opcode_and_settings[-4]), int(opcode_and_settings[-5])]
        
#         window = operation_length[opcode]
#         win = comp_string[i: i+window]
#         i+=window        
#         output = win[-1]
#         num_inputs = operation_num_inputs[opcode]
#         inputs = [win[j+1] if input_modes[j] else comp_string[win[j+1]] for j in range(operation_num_inputs[opcode])]
        
#         if(opcode == INPUT): 
#             inputs = [c_i.pop(0), 0]
            
#         if(opcode == OUTPUT): 
#             return comp_string[output]
#             continue
            
#         if(opcode in JUMPS):
#             i = {
#                 JUMP_IF_TRUE: inputs[1] if inputs[0] != 0 else i,
#                 JUMP_IF_FALSE: inputs[1] if inputs[0] == 0 else i, 
#             }[opcode]
            
#         else:
#             comp_string[output] = {
#                 ADD: (inputs[0] + inputs[1]),
#                 MULT: (inputs[0] * inputs[1]),
#                 INPUT: inputs[0],
#                 LESS_THAN: 1 if inputs[0] < inputs[1] else 0, 
#                 EQUALS: 1 if inputs[0] == inputs[1] else 0 
#             }[opcode]

## part one

In [60]:
%%time

amp_pos = list(itertools.permutations(list(range(NUM_AMPS))))

max_val = 0

for perm in amp_pos:
    inputs = [perm[0], 0]
    for i in range(NUM_AMPS):
        inputs = [perm[i+1] if i+1<NUM_AMPS else 0, intcomp.run_computer(amp_prog[:], inputs)[0]]
        if(inputs[1]>max_val):
            max_val = inputs[1]
        
print_result(max_val)

13
31
124
501
12540
13
31
124
3115
12465
13
31
129
516
12915
13
31
129
3240
12960
13
31
790
3160
12645
13
31
790
3165
12660
13
52
109
441
11040
13
52
109
2740
10965
13
52
213
431
10790
13
52
213
5340
10685
13
52
1315
2635
10545
13
52
1315
5265
10535
13
57
119
476
11915
13
57
119
2990
11960
13
57
228
461
11540
13
57
228
5715
11435
13
57
1440
2885
11540
13
57
1440
5760
11525
13
340
685
2740
10965
13
340
685
2745
10980
13
340
1360
2725
10905
13
340
1360
5445
10895
13
340
1365
2735
10940
13
340
1365
5460
10925
5
53
212
853
21340
5
53
212
5315
21265
5
53
217
868
21715
5
53
217
5440
21760
5
53
1340
5360
21445
5
53
1340
5365
21460
5
20
173
697
17440
5
20
173
4340
17365
5
20
85
693
17340
5
20
85
2140
17133
5
20
515
4133
16537
5
20
515
2065
16533
5
25
213
852
21315
5
25
213
5340
21360
5
25
100
813
20340
5
25
100
2515
20133
5
25
640
5133
20532
5
25
640
2560
20493
5
140
1133
4532
18133
5
140
1133
4537
18148
5
140
560
4493
17977
5
140
560
2245
17973
5
140
565
4533
18132
5
140
565
2260
18093
0
13
3

## part two

In [47]:
%%time

#amp_prog = [3,26,1001,26,-4,26,3,27,1002,27,2,27,1,27,26,27,4,27,1001,28,-1,28,1005,28,6,99,0,0,5]

def run_computer(ip, comp_string, c_i):
    while(True):
        opcode_and_settings = "0000"+str(comp_string[ip])
        opcode = int(opcode_and_settings[-2:])
        if(opcode == END): 
            return -1, ip, c_i
        
        input_modes = [int(opcode_and_settings[-3]), int(opcode_and_settings[-4]), int(opcode_and_settings[-5])]
        
        window = operation_length[opcode]
        win = comp_string[ip: ip+window]
        ip += window
        
        output = win[-1]
        num_inputs = operation_num_inputs[opcode]
        inputs_prog = [win[i+1] if input_modes[i] else comp_string[win[i+1]] for i in range(operation_num_inputs[opcode])] 
        if(opcode == INPUT): 
            inputs_prog = [c_i.pop(0), 0]
        if(opcode == OUTPUT): 
            return comp_string[output], ip, c_i
            
        if(opcode in JUMPS):
            ip = {
                JUMP_IF_TRUE: inputs_prog[1] if inputs_prog[0] != 0 else ip,
                JUMP_IF_FALSE: inputs_prog[1] if inputs_prog[0] == 0 else ip, 
            }[opcode]
        else:
            comp_string[output] = {
                ADD: (inputs_prog[0] + inputs_prog[1]),
                MULT: (inputs_prog[0] * inputs_prog[1]),
                INPUT: inputs_prog[0],
                LESS_THAN: 1 if inputs_prog[0] < inputs_prog[1] else 0, 
                EQUALS: 1 if inputs_prog[0] == inputs_prog[1] else 0 
            }[opcode]

amp_pos = list(itertools.permutations(list(range(NUM_AMPS, NUM_AMPS+5))))

signals = []
for perm in amp_pos:
    i = 0
    inputs = [[perm[j]] for j in range(NUM_AMPS)]
    inputs[0].append(0)
    last_e = 0
    amp_progs = [deepcopy(amp_prog) for j in range(NUM_AMPS)]
    ips = [0 for j in range(NUM_AMPS)]
    while(True):
        output, ips[i%NUM_AMPS],inputs[i%NUM_AMPS] = run_computer(ips[i%NUM_AMPS], amp_progs[i%NUM_AMPS], inputs[i%NUM_AMPS])
        if output == -1:
            break
        if(i%NUM_AMPS == 4):
            last_e = output
        i+=1
        inputs[i%NUM_AMPS].append(output)
    signals.append(last_e)
        
print_result(max(signals))

NameError: name 'END' is not defined

# 8

## setup 

In [52]:
img = list(get_level_input("08"))
img = list(map(int, img))
IMG_W = 25
IMG_H = 6

## part one

In [95]:
arr_img = np.array(img)
lay_img = np.reshape(arr_img, (-1, IMG_H, IMG_W))

print(lay_img.shape)

max_zero_index = np.argmin([(lay_img[i,:,:]==0).sum() for i in range(lay_img.shape[0])])

max_zero_layer = lay_img[max_zero_index, :, :]
NUM_1_BY_2 = (max_zero_layer == 1).sum() * (max_zero_layer == 2).sum()
print(NUM_1_BY_2)

(100, 6, 25)
1360


## part two

In [103]:
BLACK = 0 
WHITE = 1 
TRANS = 2

arr_img = np.array(img)
lay_img = np.reshape(arr_img, (-1, IMG_H, IMG_W))

final_image = np.full((IMG_H, IMG_W), 2)

spears = [lay_img[:, col, row] for col in range(lay_img.shape[1]) for row in range(lay_img.shape[2])]

first_non_transparent = [spear[np.argmax(spear!=2)] for spear in spears]
message = np.array(first_non_transparent).reshape((IMG_H, IMG_W))
for row in message:
    row = np.where(row==0, '.', row) 
    row = np.where(row=='1', 'X', row) 
    print("".join(row))

XXXX.XXX..X..X..XX..XXX..
X....X..X.X..X.X..X.X..X.
XXX..X..X.X..X.X..X.X..X.
X....XXX..X..X.XXXX.XXX..
X....X....X..X.X..X.X.X..
X....X.....XX..X..X.X..X.


# 9

## setup

In [13]:
# GET LEVEL INPUT
amp_prog = get_level_input("09")
amp_prog = list(map(int, amp_prog.split(',')))
amp_prog = tuple(amp_prog)

# OPCODES
ADD = 1
MULT = 2
INPUT = 3
OUTPUT = 4
JUMP_IF_TRUE = 5
JUMP_IF_FALSE = 6 
LESS_THAN = 7 
EQUALS = 8
ADJUST_REL_BASE = 9
END = 99

# OPCODE to OPNAME

OPNAME = {
    1: 'add',
    2: 'mult',
    3: 'input',
    4: 'output',
    5: 'jump if true',
    6: 'jump if false', 
    7: 'less than', 
    8: 'equal',
    9: 'adjust rel base',
    99: 'end'    
}
# MODES
POS_MODE = 0
IM_MODE = 1
REL_MODE = 2

MODE_NAME = {
    0: 'POSITION',
    1: 'IMMEDIATE',
    2: 'RELATIVE'
}
# OP_FEATURES
operation_length = {
    ADD: 4, 
    MULT: 4, 
    INPUT: 2, 
    OUTPUT: 2, 
    END: 1, 
    JUMP_IF_TRUE: 3, 
    JUMP_IF_FALSE: 3, 
    LESS_THAN: 4, 
    EQUALS: 4,
    ADJUST_REL_BASE: 2
}

operation_num_inputs = {
    ADD: 3, 
    MULT: 3, 
    INPUT: 1, 
    OUTPUT: 1, 
    JUMP_IF_TRUE: 2, 
    JUMP_IF_FALSE: 2, 
    LESS_THAN: 3, 
    EQUALS: 3,
    ADJUST_REL_BASE: 1
}

# JUMP SET
JUMPS = { JUMP_IF_TRUE, JUMP_IF_FALSE }

WRITES = {ADD, MULT, INPUT, LESS_THAN, EQUALS}

# DISPLAY FUNCITONS
def print_computer(comp_string):
    for i in range(0, len(comp_string), 10):
        print(f"[{i}]    {comp_string[i:i+10]}")
 
def print_step(opcode, win, input_modes, inputs, output, rel_base):
    print(f"{OPNAME[opcode]}")
    print(f"win: {win}")
    print(f"input modes: {[MODE_NAME[i] for i in input_modes]}")
    print(f"inputs: {inputs}")    
    print(f"output: {output}")
    print(f"rel base: {rel_base}\n")
          
# TESTS
tests = {
    (109,1,204,-1,1001,100,1,100,1008,100,16,101,1006,101,0,99): [109,1,204,-1,1001,100,1,100,1008,100,16,101,1006,101,0,99],
    (109, -1, 4, 1, 99): [-1], 
    (109, -1, 104, 1, 99): [1],
    (109, -1, 204, 1, 99): [109],
    (109, 1, 9, 2, 204, -6, 99): [204],
    (109, 1, 109, 9, 204, -6, 99): [204],
    (109, 1, 209, -1, 204, -106, 99): [204], 
    (109, 1, 3, 3, 204, 2, 99): [1000], #input value
    (109, 1, 203, 2, 204, 2, 99): [1000], # input value
    (104,1125899906842624,99): [1125899906842624], 
}

def run_computer(comp_string, c_i, ip = 0, rel_base = 0):
    while(True):
        opcode_and_settings = str(comp_string[ip]).zfill(5)
        opcode = int(opcode_and_settings[-2:])
        
        if(opcode == END): 
            return 'ENDED GRACEFULLY'
        
        input_modes = [int(opcode_and_settings[-3]), int(opcode_and_settings[-4]), int(opcode_and_settings[-5])]
        
        window = operation_length[opcode]
        win = comp_string[ip: ip+window]
        ip += window
        
        num_inputs = operation_num_inputs[opcode]
        inputs_prog = [win[i+1] if input_modes[i]==IM_MODE
            else comp_string[rel_base+win[i+1]] if input_modes[i]==REL_MODE
            else comp_string[win[i+1]]
            for i in range(operation_num_inputs[opcode]-1)]
        
        # WRITE OPERATIONS LOCATIONS (OUTPUT) NEVER IN POSITION MODE
        if(opcode in WRITES):
            inputs_prog.append(rel_base+win[-1] if input_modes[operation_num_inputs[opcode]-1]==REL_MODE else win[-1])
        else:
            inputs_prog.append(win[-1] if input_modes[operation_num_inputs[opcode]-1]==IM_MODE else comp_string[rel_base+win[-1]] if input_modes[operation_num_inputs[opcode]-1]==REL_MODE else comp_string[win[-1]]) 
         
        if(opcode == INPUT): 
            inputs_prog.insert(0,c_i.pop(0))
        
        if(opcode == OUTPUT): 
            print(inputs_prog[-1])
            continue
    
        if(opcode in JUMPS):
            ip = {
                JUMP_IF_TRUE: inputs_prog[1] if inputs_prog[0] != 0 else ip,
                JUMP_IF_FALSE: inputs_prog[1] if inputs_prog[0] == 0 else ip, 
            }[opcode]
            
        elif(opcode == ADJUST_REL_BASE):
            rel_base = {
                ADJUST_REL_BASE: rel_base+inputs_prog[0]
            }[opcode]
            
        else:
            comp_string[inputs_prog[-1]] = {
                ADD: (inputs_prog[0] + inputs_prog[1]),
                MULT: (inputs_prog[0] * inputs_prog[1]),
                INPUT: inputs_prog[0],
                LESS_THAN: 1 if inputs_prog[0] < inputs_prog[1] else 0, 
                EQUALS: 1 if inputs_prog[0] == inputs_prog[1] else 0 
            }[opcode]


## part one

In [14]:
%%time
BOOST_INPUT = 1

run_computer(list(amp_prog)+[0 for x in range(9999)], [BOOST_INPUT])

2204990589
CPU times: user 1.51 ms, sys: 86 µs, total: 1.6 ms
Wall time: 1.59 ms


'ENDED GRACEFULLY'

## part two 

In [15]:
%%time
SENSOR_MODE = 2

run_computer(list(amp_prog)+[0 for x in range(9999)], [SENSOR_MODE])

50008
CPU times: user 1.21 s, sys: 5.68 ms, total: 1.21 s
Wall time: 1.22 s


'ENDED GRACEFULLY'

# 10 

## setup 

In [8]:
asteroids = get_level_input("10").splitlines()
asteroid_grid = np.array([np.array(list(belt)) for belt in asteroids])
asteroid_set = frozenset(zip(*np.where(asteroid_grid == '#')[::-1]))

X = 0
Y = 1

sort_order = {
    '+': 1,
    '-': -1
}

def calc_slope(p1, p2):
    if(p1[X]-p2[X] == 0):
        return (np.inf, '+' if (p1[Y]-p2[Y]) < 0 else '-')
    if(p1[Y]-p2[Y]==0):
        return (0, '-' if (p1[X]-p2[X]) > 0 else '+')
    return ((p1[Y]-p2[Y])/(p1[X]-p2[X]), '-' if (p1[X]-p2[X]) > 0 else '+')

## part one

In [10]:
%%time

visible_sets = [(asteroid, set(map(calc_slope, itertools.repeat(asteroid), asteroid_set-{asteroid}))) for asteroid in asteroid_set]
  
monitoring_location, visible_asteroids = max(visible_sets, key = lambda x: len(x[1]))
print_result(f"{len(visible_asteroids)} asteroids visible at {monitoring_location}")

'RESULT: 284 asteroids visible at (20, 19)'

'TIME............................................................'
CPU times: user 259 ms, sys: 6.86 ms, total: 265 ms
Wall time: 266 ms


## part two 

In [24]:
%%time
MONITORING_STATION = (20, 19)
REMAINING_ASTEROIDS = asteroid_set-{MONITORING_STATION}

def l1(p1, p2):
    return abs(p1[X]-p2[X])+abs(p1[Y]-p2[Y])

def calc_slope_and_dist(p1, p2):
    dist = l1(p1,p2)
    if(p1[X]-p2[X] == 0):
        return (np.inf, '+' if (p1[Y]-p2[Y]) > 0 else '-', dist)
    if(p1[Y]-p2[Y]==0):
        return (0, '-' if (p1[X]-p2[X]) > 0 else '+', dist)
    return (-(p1[Y]-p2[Y])/(p1[X]-p2[X]), '-' if (p1[X]-p2[X]) > 0 else '+', dist)

visible_list = [(asteroid, calc_slope_and_dist(MONITORING_STATION, asteroid)) for asteroid in REMAINING_ASTEROIDS]
visible_list.sort(key=lambda k: (sort_order[k[1][1]], k[1][0], -k[1][2]), reverse=True)

visible_stacked = [list(g) for k, g in itertools.groupby(visible_list, key=lambda k: k[1][:1])]
two_hundredth_vape = visible_stacked[199][0][0]

output_val = two_hundredth_vape[0]*100+two_hundredth_vape[1]

print_result(output_val)

'RESULT: 404'

'TIME............................................................'
CPU times: user 2.83 ms, sys: 363 µs, total: 3.19 ms
Wall time: 2.87 ms


# 11 

## setup 

In [2]:
paint_prog = get_level_input("11")
paint_prog = list(map(int, paint_prog.split(',')))
paint_prog = tuple(paint_prog)
import incode_comp/intcomp

SyntaxError: invalid syntax (<ipython-input-2-428b7f29a61e>, line 4)